In [ ]:
import cv2 as cv
import numpy as np
from PyQt5.QtWidgets import *
import sys    

# [1] PyQt5 메인 윈도우 클래스
class SpecialEffect(QMainWindow):
    def __init__(self):
        super().__init__()  # QMainWindow 부모 초기화
        self.setWindowTitle('사진 특수 효과')  # 창 제목
        self.setGeometry(200, 200, 800, 200)  # 위치 + 크기

        # [2] 버튼 6개 + 콤보박스 + 라벨 생성
        pictureButton = QPushButton('사진 읽기', self)
        embossButton = QPushButton('엠보싱', self)
        cartoonButton = QPushButton('카툰', self)
        sketchButton = QPushButton('연필 스케치', self)
        oilButton = QPushButton('유화', self)
        saveButton = QPushButton('저장하기', self)

        self.pickCombo = QComboBox(self)
        self.pickCombo.addItems(['엠보싱', '카툰', '연필 스케치(명암)', '연필 스케치(컬러)', '유화'])

        quitButton = QPushButton('나가기', self)
        self.label = QLabel('환영합니다!', self)

        # [3] 버튼과 콤보박스, 라벨의 위치 지정
        pictureButton.setGeometry(10, 10, 100, 30)
        embossButton.setGeometry(110, 10, 100, 30)
        cartoonButton.setGeometry(210, 10, 100, 30)
        sketchButton.setGeometry(310, 10, 100, 30)
        oilButton.setGeometry(410, 10, 100, 30)
        saveButton.setGeometry(510, 10, 100, 30)
        self.pickCombo.setGeometry(510, 40, 110, 30)
        quitButton.setGeometry(620, 10, 100, 30)
        self.label.setGeometry(10, 40, 500, 170)

        # [4] 각 버튼 클릭 시 실행될 함수 연결
        pictureButton.clicked.connect(self.pictureOpenFunction)
        embossButton.clicked.connect(self.embossFunction)
        cartoonButton.clicked.connect(self.cartoonFunction)
        sketchButton.clicked.connect(self.sketchFunction)
        oilButton.clicked.connect(self.oilFunction)
        saveButton.clicked.connect(self.saveFunction)
        quitButton.clicked.connect(self.quitFunction)

    # [5] 이미지 파일 열기 함수
    def pictureOpenFunction(self):
        fname = QFileDialog.getOpenFileName(self, '사진 읽기', './')
        self.img = cv.imread(fname[0])
        if self.img is None:
            sys.exit('파일을 찾을 수 없습니다.')

        cv.imshow('Painting', self.img)

    # [6] 엠보싱 필터 적용 함수
    def embossFunction(self):
        # 엠보싱 커널
        femboss = np.array([[-1.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0],
                            [0.0, 0.0, 1.0]])

        gray = cv.cvtColor(self.img, cv.COLOR_BGR2GRAY)  # 흑백 변환
        gray16 = np.int16(gray)                          # int16로 변환 (음수값 포함)
        # filter2D로 커널 적용 후 128로 이동(중앙값) → clip으로 0~255 제한 → uint8로 변환
        self.emboss = np.uint8(np.clip(cv.filter2D(gray16, -1, femboss) + 128, 0, 255))

        cv.imshow('Emboss', self.emboss)

    # [7] 카툰 효과 함수
    def cartoonFunction(self):
        # OpenCV 내장 stylization 사용
        self.cartoon = cv.stylization(self.img, sigma_s=60, sigma_r=0.45)
        cv.imshow('Cartoon', self.cartoon)

    # [8] 연필 스케치 효과 함수
    def sketchFunction(self):
        # pencilSketch는 흑백과 컬러 버전 모두 리턴함
        self.sketch_gray, self.sketch_color = cv.pencilSketch(
            self.img, sigma_s=60, sigma_r=0.07, shade_factor=0.02
        )
        cv.imshow('Pencil sketch(gray)', self.sketch_gray)
        cv.imshow('Pencil sketch(color)', self.sketch_color)

    # [9] 유화 효과 함수
    def oilFunction(self):
        # oilPainting 함수는 OpenCV xphoto 모듈에 있음
        self.oil = cv.xphoto.oilPainting(self.img, 10, 1, cv.COLOR_BGR2Lab)
        cv.imshow('Oil painting', self.oil)

    # [10] 이미지 저장 함수
    def saveFunction(self):
        fname = QFileDialog.getSaveFileName(self, '파일 저장', './')
        i = self.pickCombo.currentIndex()  # 콤보박스에서 선택된 효과 인덱스 가져오기

        # 선택된 효과 결과 이미지 저장
        if i == 0:
            cv.imwrite(fname[0], self.emboss)
        elif i == 1:
            cv.imwrite(fname[0], self.cartoon)
        elif i == 2:
            cv.imwrite(fname[0], self.sketch_gray)
        elif i == 3:
            cv.imwrite(fname[0], self.sketch_color)
        elif i == 4:
            cv.imwrite(fname[0], self.oil)

    # [11] 종료 함수
    def quitFunction(self):
        cv.destroyAllWindows()  # OpenCV 창 닫기
        self.close()            # PyQt 윈도우 닫기

# [12] PyQt5 앱 실행
app = QApplication(sys.argv)
win = SpecialEffect()
win.show()
app.exec_()

# SpecialEffect 프로그램 핵심 개념 정리

## 1) OpenCV 커널 필터 (엠보싱)
- filter2D(): 사용자 정의 커널로 이미지 필터링
- 왜 np.int16로 바꾸는지: filter2D는 음수 값이 나올 수 있어서 음수로 표현해야 함
- 엠보싱 후 +128: 음수를 밝기로 변환하기 위해 Shift

## 2) stylization(), pencilSketch(), oilPainting()
- OpenCV에서 제공하는 고급 스타일 함수
- `stylization()`: 카툰 효과
- `pencilSketch()`: 흑백/컬러 연필 스케치 동시에 리턴
- `oilPainting()`: xphoto 모듈에 있음 (OpenCV-contrib 설치 필요)

## 3) PyQt5 + OpenCV 연동
- PyQt5 버튼에 함수 연결: Signal-Slot
- QFileDialog: 파일 열기/저장 다이얼로그
- QLabel: 상태 메시지 출력
- QComboBox: 저장할 효과 선택

## 4) 데이터 타입과 범위 처리
- filter2D 결과는 음수가 될 수 있으므로 int16 사용 → 결과에 +128 → np.clip으로 0~255 → uint8
- 이미지 처리에서 dtype(데이터 타입) 이해가 중요

---